In [1]:
import notebook_import_hook
notebook_import_hook.attach()
from helper.helper import *

In [2]:
def wmean(a, weights, axis=0):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as `a` or else broadcastable to `a`
    Weighted mean, with `a.shape` minus the {observations} dimension.'''
    w = weights + only_nans(a) #broadcast(weights, only_nans(a), '+')
    return (a * w).sum(axis=axis) / w.sum(axis=axis)
pd.DataFrame.wmean = wmean
pd.Series.wmean = wmean

In [2]:
def wvar(a, weights, ddof=1, axis=0):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as a
                ddof: mimics ddof of unweighted stats calculation in scipy
    Weighted variance (reduced 1 dimension).
    Every +1 on ddof corresponds to weights.mean() deducted from the denominator.'''
    w = weights + only_nans(a)
    w_sum = w.sum(axis=axis)
    a_mean = (a * w).sum(axis=axis) / w_sum
    return (w * (a - a_mean) ** 2).sum(axis=axis) / (w_sum - w.mean(axis=axis) * ddof)
pd.DataFrame.wvar = wvar
pd.Series.wvar = wvar

def wstd(a, weights, ddof=1, axis=0):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as a
                ddof: mimics ddof of unweighted stats calculation in scipy
    Weighted standard deviation (reduced 1 dimension).
    Every +1 on ddof corresponds to weights.mean() deducted from the denominator.'''
    return np.sqrt(wvar(a, weights, ddof=ddof, axis=axis))
pd.DataFrame.wstd = wstd
pd.Series.wstd = wstd

In [ ]:
def weighted_quantile_transform(X, weights, range=None):
    '''Params - X: {observations}x{features} `DataFrame`/`ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as `X` or else broadcastable to `X`
                range: 2-`tuple`-like indicating the lower and upper range of resulting quantiles, e.g. [0,1]->range
    Weighted quantiles, with `X.shape` minus the {observations} dimension.'''
    W = weights + only_nans(X)
    try:
        X.replace(np.nan, np.inf, inplace=True)
        Wsort = W.values_sort(X)
        rev = X.values.argsort(axis=0).argsort(axis=0)
    finally:
        X.replace(np.inf, np.nan, inplace=True)
    wavg, wsum = W.mean(), W.sum()
    Wavg, Wsum = wavg.values[np.newaxis, :], wsum.values[np.newaxis, :]
    pct = (Wavg / 2 + Wsort.cumsum(axis=0) - Wsort / 2) / (Wsum + Wavg)
    pct = pct[rev, np.arange(pct.shape[1])]
    if range:
        pct *= range[1] - range[0]
        pct -= (range[1] - range[0]) / 2
    return pd.DataFrame(pct, index=X.index, columns=X.columns)
pd.DataFrame.wqtl = weighted_quantile_transform

In [245]:
def wmedian(a, weights):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as a
    Weighted quantiles, with `X.shape` minus the {observations} dimension.'''
    weights = weights + only_nans(a)
    orig_type = None
    if not isinstance(a, np.ndarray):
        orig_type = a.min(axis=0) * 0
        a = np.ma.array(a.values, mask=np.isnan(a.values))
    w = weights
    if not isinstance(w, np.ndarray):
        w = np.ma.array(w.values, mask=np.isnan(w.values))
    order = a.argsort(axis=0)
    ordereda = a[(order,) + ((np.arange(a.shape[1]),) if len(a.shape) > 1 else ())]
    orderedw = w[(order,) + ((np.arange(a.shape[1]),) if len(a.shape) > 1 else ())]
    cumdist = orderedw.cumsum(axis=0)
    half = cumdist.max(axis=0) / 2
    atleast = np.ma.array(ordereda, mask=(cumdist < half))
    median = atleast.min(axis=0)
    median = np.ma.filled(median, np.nan)
    if orig_type is not None:
        return median + orig_type
    return median
pd.DataFrame.wmedian = wmedian

In [ ]:
def wskew(a, weights, ddof=0):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                    if `ndarray`, must not contain NaN
                weights: weights of same shape as a
                ddof: mimics ddof of unweighted stats calculation in scipy
    Weighted skew. Every +1 on ddof corresponds to weights.mean() deducted from the denominator.'''
    pass